<a href="https://colab.research.google.com/github/imj2185/csc_movement/blob/master/csc_movement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 6.6 MB/s eta 0:00:00


In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import math
import argparse
import xlsxwriter

In [5]:
torch.set_default_dtype(torch.float64)

In [6]:
class DEDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, scale_data=False):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            #Apply scaling if necessary
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(25, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.layers(x)

In [11]:
exp_number=1
val_number=4
val_count=51
sample_name='Xylem10%'
train=True
test=False

mlp = MLP()

Training

In [10]:
torch.manual_seed(42)
    
# Load dataset
DE_list = ['DE0', 'DE50', 'DE100']
xls = pd.ExcelFile('./train_' + str(exp_number)+ '/train.xlsx')

X = []
y = []

for de in DE_list:
    azimuth = []
    df1 = pd.read_excel(xls, de)

    tilt_angle_1 = [i*15 for i in range(12)]
    tilt_angel_2 = [i*(-15) for i in range(1, 13)]

    tilt_angle = tilt_angle_1 + tilt_angel_2

    for i in tilt_angle:
        azimuth.append(df1[i].values)

    tilt_angel_2 = [(i*(-15) + 360) for i in range(1, 13)]
    tilt_angle = tilt_angle_1 + tilt_angel_2

    for i in range(len(tilt_angle)):
        tilt_angle[i] = [((tilt_angle[i] * math.pi) / 180.0), float(de[2:]) / 100.0]

    X = X + azimuth
    y = y + tilt_angle


X = np.array(X)
y = np.array(y)
y = y.astype(float)

# Prepare dataset
dataset = DEDataset(X, y)
trainloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Define the loss function and optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-3)

best_loss = 100.0
# Run the training loop
for epoch in range(0, 15000): # 5 epochs at maximum

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader):

    # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.double(), targets.double()
        targets = targets.reshape((targets.shape[0], 2))

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Zero the gradients
        optimizer.zero_grad()

        current_loss += loss.item()
        
    if epoch % 100 == 0:
        cur_loss = (current_loss / (100 * len(trainloader)))
        print('Loss after epoch %5d: %.12f' % (epoch, cur_loss))
        if best_loss > cur_loss:
            best_loss = cur_loss
            torch.save(mlp.state_dict(), './train_' + str(exp_number)+ '/best_checkpoint.bin')

    current_loss = 0.0

# Process is complete.
print('Training process has finished.')

Loss after epoch     0: 0.017097230884
Loss after epoch   100: 0.004477044289
Loss after epoch   200: 0.003150767880
Loss after epoch   300: 0.002124932224
Loss after epoch   400: 0.001539843420
Loss after epoch   500: 0.001802785576
Loss after epoch   600: 0.001263392926
Loss after epoch   700: 0.000990343782
Loss after epoch   800: 0.001076941149
Loss after epoch   900: 0.000824447324
Loss after epoch  1000: 0.000782863774
Loss after epoch  1100: 0.000742039187
Loss after epoch  1200: 0.000756213441
Loss after epoch  1300: 0.000678497958
Loss after epoch  1400: 0.000509404041
Loss after epoch  1500: 0.000704871966
Loss after epoch  1600: 0.000466985961
Loss after epoch  1700: 0.000314606698
Loss after epoch  1800: 0.000584981609
Loss after epoch  1900: 0.000460089320
Loss after epoch  2000: 0.000638536283
Loss after epoch  2100: 0.000432042987
Loss after epoch  2200: 0.000368660332
Loss after epoch  2300: 0.000481917680
Loss after epoch  2400: 0.000451512116
Loss after epoch  2500: 0

In [12]:
xls = pd.ExcelFile('./train_' + str(exp_number)+ '/val' + str(val_number)+ '.xlsx')
df2 = pd.read_excel(xls, sample_name)

sample_count = val_count
y = [0 for i in range(sample_count)]
X = []

for i in range(sample_count):
    X.append(df2[i].values)

X = np.array(X)
y = np.array(y)
y = y.astype(float)

dataset = DEDataset(X, y)
testloader = DataLoader(dataset, batch_size=val_count, shuffle=False)

mlp.load_state_dict(torch.load('./train_' + str(exp_number)+ '/best_checkpoint.bin'))
mlp.eval()

workbook = xlsxwriter.Workbook('train_' + str(exp_number) + '_' + sample_name + '.xlsx')
worksheet = workbook.add_worksheet(sample_name)
row = 0
col = 0

for i, data in enumerate(testloader):
    # Get and prepare inputs
    inputs, targets = data
    inputs, targets = inputs.double(), targets.double()
    targets = targets.reshape((targets.shape[0], 1))

    # Perform forward pass
    outputs = mlp(inputs)
    print(inputs)
    angles = (outputs[:, :1] * 180.0) / math.pi # angle
    des = (outputs[:, 1:2]) * 100.0             # DE

    for i, tup in enumerate(zip(angles, des)):
        print(tup[0].item(), tup[1].item())
        worksheet.write(row, col, tup[0].item())
        worksheet.write(row, col + 1, tup[1].item())
        row += 1

workbook.close()

tensor([[1.0593, 0.8839, 0.7276,  ..., 0.4865, 0.7015, 1.0593],
        [1.0857, 0.7766, 0.6916,  ..., 0.4349, 0.7770, 1.0857],
        [0.9945, 0.7160, 0.7062,  ..., 0.3932, 0.7180, 0.9945],
        ...,
        [0.6937, 1.0862, 0.6308,  ..., 0.2201, 0.7237, 0.6937],
        [0.7006, 1.1319, 0.6226,  ..., 0.2130, 0.7280, 0.7006],
        [0.7602, 1.2292, 0.6170,  ..., 0.2164, 0.6756, 0.7602]])
148.7681371946241 152.66871698302026
149.87833888222343 159.11775485861227
149.17435275915884 158.85976373287065
148.74481904784233 160.29568312663213
149.50540413515358 158.62151380049062
149.39510503656572 158.04761197611643
148.25749342118422 163.13936431136125
147.48560388260864 163.24638431944297
147.81827229890942 163.21161276584718
147.57626683911195 161.73560506065468
147.2842981573963 158.36811359273418
146.82067058519408 158.42013546569777
145.9749614559384 159.89124079070828
144.19218500676266 161.5197844456641
145.46201019341206 159.4202431849287
144.53629304558288 162.22301033939942